In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
pd.options.display.max_columns=100
pd.options.display.max_rows=100
from encoding import FreqeuncyEncoding
from sklearn.externals import joblib
import datetime
from catboost_custom_estimator_regressor import Estimator
from custom_fold_generator import CustomFolds

In [2]:
train=pd.read_csv('train_1.csv')
test=pd.read_csv('test_1.csv')
target=train.amount_spent_per_room_night_scaled

In [3]:
df=pd.read_csv('temp_df.csv')

In [4]:
df['cat_1']= df['resort_id'].astype('str')+'_'+df['memberid'].astype('str')
df['cat_2']= df['resort_id'].astype('str')+'_'+df['state_code_residence'].astype('str')
df['cat_3']= df['resort_id'].astype('str')+'_'+df['state_code_resort'].astype('str')
df['cat_4']= df['resort_id'].astype('str')+'_'+df['checkin_date'].astype('str')
df['cat_5']= df['resort_id'].astype('str')+'_'+df['checkout_date'].astype('str')
df['cat_6']= df['state_code_residence'].astype('str')+'_'+df['state_code_residence'].astype('str')
df['cat_7']= df['memberid'].astype('str')+'_'+df['checkin_date'].astype('str')
df['cat_8']= df['memberid'].astype('str')+'_'+df['checkout_date'].astype('str')
df['cat_9']= df['memberid'].astype('str')+'_'+df['cluster_code'].astype('str')
df['cat_10']= df['memberid'].astype('str')+'_'+df['booking_date'].astype('str')


In [5]:
cat_cols=['booking_date', 'checkin_date', 'checkout_date', 'channel_code', 
          'main_product_code', 'persontravellingid',
          'resort_region_code', 'resort_type_code', 'room_type_booked_code', 
          'season_holidayed_code', 'state_code_residence', 'state_code_resort', 
          'member_age_buckets', 'booking_type_code', 'cluster_code', 'reservationstatusid_code',
          'resort_id', 'booking_mon','booking_day', 'booking_year', 'booking_dow', 
          'booking_week_no', 'booking_day_of_year', 'booking_quarter', 
          'booking_week_of_yr', 'checkin_mon', 'checkin_day', 'checkin_year', 'checkin_dow',
          'checkin_week_no', 'checkin_day_of_year', 'checkin_quarter', 'checkin_week_of_yr',
         'cat_2','cat_3','cat_6'
         ]

cat_cols2=['memberid','cat_1','cat_7','cat_8','cat_9','cat_10','cat_4','cat_5']

In [6]:
num_cols=['numberofadults', 'numberofchildren','roomnights','total_pax','checkin_out_delta','booking_checkin_delta',
         'booking_checkout_delta','child_adult_ratio','num_single_rm_nts_adults','num_single_rm_nts_children',
         'adult_Frac','child_Frac','pax_rm_nts','is_children','cont1','delta_rmnts_checin_in_out_delta',
         'calc_pax_rm_nts',
         'pax_cum_mean_state_ind', 'checkin_out_delta_cum_mean_resort','visit_number_resort',
          'visit_number_state_ind','visit_number_state_res','pax_cum_mean_resort']

In [7]:
print len(cat_cols), len(num_cols)

36 23


In [8]:
cat_inds=[df.columns.get_loc(i) for i in cat_cols]

In [9]:
cat_total=cat_cols+cat_cols2

In [10]:
fe=FreqeuncyEncoding(categorical_columns=cat_total,normalize=False,return_df=True)
df=fe.fit_transform(df)

In [11]:
train=df[df.reservation_id.isin(train.reservation_id)]
test=df[df.reservation_id.isin(test.reservation_id)]
train.drop('reservation_id',axis=1,inplace=True)
test.drop('reservation_id',axis=1,inplace=True)
print train.shape, test.shape

(341424, 118) (146765, 118)


/home/ubuntu/anaconda2/lib/python2.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [12]:
cat_inds=[train.columns.get_loc(i) for i in cat_total]

In [13]:
from catboost import CatBoostRegressor

In [14]:
cat=CatBoostRegressor(eval_metric='RMSE',n_estimators=25000,od_type='Iter',od_wait=400,colsample_bylevel=0.4,max_depth=6,
                   learning_rate=0.02)
mod=Estimator(model=cat,early_stopping_rounds=200,random_state=100,n_jobs=-1,shuffle=True,
              validation_scheme='KFold',
          n_splits=5,categorical_features_indices=cat_inds)

In [15]:
%%time
oof=mod.fit_transform(train.values,target.values)

0:	learn: 7.6436887	test: 7.6480895	best: 7.6480895 (0)	total: 192ms	remaining: 1h 19m 51s
100:	learn: 1.4427180	test: 1.4456623	best: 1.4456623 (100)	total: 11.5s	remaining: 47m 21s
200:	learn: 0.9974695	test: 0.9970474	best: 0.9970474 (200)	total: 27.1s	remaining: 55m 45s
300:	learn: 0.9773834	test: 0.9766081	best: 0.9766081 (300)	total: 43.8s	remaining: 59m 52s
400:	learn: 0.9730177	test: 0.9724583	best: 0.9724583 (400)	total: 1m	remaining: 1h 2m 5s
500:	learn: 0.9707313	test: 0.9705711	best: 0.9705711 (500)	total: 1m 17s	remaining: 1h 3m 4s
600:	learn: 0.9692349	test: 0.9694808	best: 0.9694808 (600)	total: 1m 33s	remaining: 1h 3m 16s
700:	learn: 0.9680092	test: 0.9687656	best: 0.9687656 (700)	total: 1m 48s	remaining: 1h 2m 55s
800:	learn: 0.9668614	test: 0.9681240	best: 0.9681240 (800)	total: 2m 4s	remaining: 1h 2m 32s
900:	learn: 0.9659255	test: 0.9676783	best: 0.9676783 (900)	total: 2m 19s	remaining: 1h 2m 19s
1000:	learn: 0.9650686	test: 0.9671696	best: 0.9671696 (1000)	total: 2

8700:	learn: 0.9342930	test: 0.9620405	best: 0.9620388 (8676)	total: 24m 41s	remaining: 46m 15s
8800:	learn: 0.9339283	test: 0.9620055	best: 0.9620029 (8775)	total: 24m 59s	remaining: 46m
8900:	learn: 0.9335904	test: 0.9619773	best: 0.9619754 (8898)	total: 25m 16s	remaining: 45m 43s
9000:	learn: 0.9332672	test: 0.9619823	best: 0.9619689 (8944)	total: 25m 34s	remaining: 45m 27s
9100:	learn: 0.9329309	test: 0.9619712	best: 0.9619689 (8944)	total: 25m 52s	remaining: 45m 12s
9200:	learn: 0.9326056	test: 0.9619443	best: 0.9619423 (9190)	total: 26m 10s	remaining: 44m 56s
9300:	learn: 0.9322487	test: 0.9618596	best: 0.9618596 (9300)	total: 26m 27s	remaining: 44m 39s
9400:	learn: 0.9318970	test: 0.9618296	best: 0.9618274 (9395)	total: 26m 45s	remaining: 44m 23s
9500:	learn: 0.9315476	test: 0.9618053	best: 0.9618025 (9485)	total: 27m 2s	remaining: 44m 7s
9600:	learn: 0.9311976	test: 0.9617699	best: 0.9617689 (9585)	total: 27m 19s	remaining: 43m 50s
9700:	learn: 0.9308653	test: 0.9617528	best: 0

2300:	learn: 0.9574197	test: 0.9666675	best: 0.9666666 (2299)	total: 6m 17s	remaining: 1h 2m 7s
2400:	learn: 0.9569726	test: 0.9666050	best: 0.9666040 (2392)	total: 6m 34s	remaining: 1h 1m 56s
2500:	learn: 0.9564568	test: 0.9664992	best: 0.9664984 (2499)	total: 6m 51s	remaining: 1h 1m 43s
2600:	learn: 0.9559909	test: 0.9664193	best: 0.9664189 (2599)	total: 7m 8s	remaining: 1h 1m 28s
2700:	learn: 0.9555561	test: 0.9663272	best: 0.9663272 (2700)	total: 7m 25s	remaining: 1h 1m 17s
2800:	learn: 0.9551495	test: 0.9662691	best: 0.9662691 (2800)	total: 7m 43s	remaining: 1h 1m 10s
2900:	learn: 0.9547065	test: 0.9661851	best: 0.9661851 (2900)	total: 8m	remaining: 1h 57s
3000:	learn: 0.9542977	test: 0.9661377	best: 0.9661346 (2997)	total: 8m 17s	remaining: 1h 44s
3100:	learn: 0.9538664	test: 0.9660431	best: 0.9660431 (3100)	total: 8m 34s	remaining: 1h 32s
3200:	learn: 0.9534176	test: 0.9659565	best: 0.9659541 (3195)	total: 8m 52s	remaining: 1h 25s
3300:	learn: 0.9530121	test: 0.9659193	best: 0.9

10900:	learn: 0.9257398	test: 0.9639464	best: 0.9639464 (10900)	total: 30m 15s	remaining: 39m 7s
11000:	learn: 0.9253825	test: 0.9639592	best: 0.9639419 (10905)	total: 30m 31s	remaining: 38m 50s
11100:	learn: 0.9250742	test: 0.9639457	best: 0.9639419 (10905)	total: 30m 48s	remaining: 38m 34s
11200:	learn: 0.9247417	test: 0.9639268	best: 0.9639268 (11200)	total: 31m 5s	remaining: 38m 18s
11300:	learn: 0.9243771	test: 0.9638990	best: 0.9638988 (11298)	total: 31m 22s	remaining: 38m 2s
11400:	learn: 0.9240322	test: 0.9638861	best: 0.9638861 (11400)	total: 31m 39s	remaining: 37m 45s
11500:	learn: 0.9236827	test: 0.9638702	best: 0.9638687 (11442)	total: 31m 56s	remaining: 37m 29s
11600:	learn: 0.9233680	test: 0.9638677	best: 0.9638621 (11518)	total: 32m 13s	remaining: 37m 13s
11700:	learn: 0.9230460	test: 0.9638430	best: 0.9638430 (11700)	total: 32m 30s	remaining: 36m 57s
11800:	learn: 0.9227217	test: 0.9638171	best: 0.9638171 (11800)	total: 32m 48s	remaining: 36m 41s
11900:	learn: 0.9223818

5100:	learn: 0.9492471	test: 0.9545017	best: 0.9544998 (5099)	total: 13m 49s	remaining: 53m 56s
5200:	learn: 0.9488895	test: 0.9544528	best: 0.9544528 (5200)	total: 14m 5s	remaining: 53m 40s
5300:	learn: 0.9485327	test: 0.9544052	best: 0.9544052 (5300)	total: 14m 22s	remaining: 53m 23s
5400:	learn: 0.9481567	test: 0.9543552	best: 0.9543552 (5400)	total: 14m 38s	remaining: 53m 8s
5500:	learn: 0.9478052	test: 0.9543058	best: 0.9543058 (5500)	total: 14m 55s	remaining: 52m 53s
5600:	learn: 0.9474300	test: 0.9542698	best: 0.9542685 (5596)	total: 15m 11s	remaining: 52m 38s
5700:	learn: 0.9470875	test: 0.9542480	best: 0.9542479 (5699)	total: 15m 28s	remaining: 52m 23s
5800:	learn: 0.9467090	test: 0.9542063	best: 0.9542060 (5799)	total: 15m 45s	remaining: 52m 9s
5900:	learn: 0.9463125	test: 0.9541593	best: 0.9541593 (5900)	total: 16m 1s	remaining: 51m 53s
6000:	learn: 0.9459933	test: 0.9541276	best: 0.9541276 (6000)	total: 16m 18s	remaining: 51m 36s
6100:	learn: 0.9456504	test: 0.9541087	best:

13600:	learn: 0.9203324	test: 0.9528803	best: 0.9528702 (13566)	total: 37m 45s	remaining: 31m 38s
13700:	learn: 0.9199970	test: 0.9528723	best: 0.9528702 (13566)	total: 38m 3s	remaining: 31m 23s
13800:	learn: 0.9196676	test: 0.9528472	best: 0.9528467 (13799)	total: 38m 22s	remaining: 31m 8s
13900:	learn: 0.9193377	test: 0.9528634	best: 0.9528454 (13805)	total: 38m 39s	remaining: 30m 52s
14000:	learn: 0.9190377	test: 0.9528485	best: 0.9528454 (13805)	total: 38m 58s	remaining: 30m 37s
14100:	learn: 0.9187123	test: 0.9528408	best: 0.9528366 (14046)	total: 39m 16s	remaining: 30m 21s
14200:	learn: 0.9183816	test: 0.9528292	best: 0.9528269 (14189)	total: 39m 33s	remaining: 30m 5s
14300:	learn: 0.9180488	test: 0.9528218	best: 0.9528202 (14293)	total: 39m 51s	remaining: 29m 49s
14400:	learn: 0.9177399	test: 0.9528104	best: 0.9528068 (14392)	total: 40m 9s	remaining: 29m 33s
14500:	learn: 0.9174441	test: 0.9528071	best: 0.9528001 (14490)	total: 40m 26s	remaining: 29m 17s
14600:	learn: 0.9171240	

6100:	learn: 0.9407946	test: 0.9723163	best: 0.9723129 (6098)	total: 16m 25s	remaining: 50m 52s
6200:	learn: 0.9404853	test: 0.9722972	best: 0.9722969 (6196)	total: 16m 41s	remaining: 50m 36s
6300:	learn: 0.9401345	test: 0.9722663	best: 0.9722631 (6296)	total: 16m 58s	remaining: 50m 21s
6400:	learn: 0.9397797	test: 0.9722433	best: 0.9722433 (6400)	total: 17m 14s	remaining: 50m 6s
6500:	learn: 0.9394290	test: 0.9722207	best: 0.9722158 (6490)	total: 17m 30s	remaining: 49m 50s
6600:	learn: 0.9390801	test: 0.9721963	best: 0.9721957 (6599)	total: 17m 46s	remaining: 49m 33s
6700:	learn: 0.9387381	test: 0.9721718	best: 0.9721718 (6700)	total: 18m 3s	remaining: 49m 17s
6800:	learn: 0.9384011	test: 0.9721476	best: 0.9721397 (6774)	total: 18m 19s	remaining: 49m 3s
6900:	learn: 0.9380340	test: 0.9721253	best: 0.9721178 (6887)	total: 18m 35s	remaining: 48m 46s
7000:	learn: 0.9376874	test: 0.9720974	best: 0.9720974 (7000)	total: 18m 51s	remaining: 48m 29s
7100:	learn: 0.9373018	test: 0.9720714	best

14600:	learn: 0.9121196	test: 0.9709073	best: 0.9709055 (14541)	total: 39m 25s	remaining: 28m 4s
14700:	learn: 0.9118209	test: 0.9709079	best: 0.9709011 (14630)	total: 39m 42s	remaining: 27m 49s
14800:	learn: 0.9115007	test: 0.9708935	best: 0.9708935 (14800)	total: 40m	remaining: 27m 34s
14900:	learn: 0.9111818	test: 0.9708999	best: 0.9708935 (14800)	total: 40m 18s	remaining: 27m 18s
15000:	learn: 0.9108697	test: 0.9709039	best: 0.9708935 (14800)	total: 40m 35s	remaining: 27m 3s
15100:	learn: 0.9105642	test: 0.9708989	best: 0.9708935 (14800)	total: 40m 53s	remaining: 26m 48s
15200:	learn: 0.9102641	test: 0.9708889	best: 0.9708867 (15197)	total: 41m 10s	remaining: 26m 32s
15300:	learn: 0.9099723	test: 0.9708697	best: 0.9708697 (15300)	total: 41m 28s	remaining: 26m 17s
15400:	learn: 0.9096556	test: 0.9708781	best: 0.9708693 (15305)	total: 41m 46s	remaining: 26m 2s
15500:	learn: 0.9093233	test: 0.9708580	best: 0.9708526 (15491)	total: 42m 3s	remaining: 25m 46s
15600:	learn: 0.9090313	test

6300:	learn: 0.9412848	test: 0.9684125	best: 0.9684102 (6268)	total: 17m 49s	remaining: 52m 52s
6400:	learn: 0.9409121	test: 0.9683726	best: 0.9683699 (6399)	total: 18m 6s	remaining: 52m 36s
6500:	learn: 0.9405558	test: 0.9683337	best: 0.9683315 (6498)	total: 18m 24s	remaining: 52m 22s
6600:	learn: 0.9401782	test: 0.9683104	best: 0.9683029 (6524)	total: 18m 42s	remaining: 52m 8s
6700:	learn: 0.9398307	test: 0.9683011	best: 0.9683010 (6699)	total: 19m	remaining: 51m 53s
6800:	learn: 0.9394740	test: 0.9682823	best: 0.9682823 (6800)	total: 19m 17s	remaining: 51m 37s
6900:	learn: 0.9391257	test: 0.9682341	best: 0.9682317 (6887)	total: 19m 35s	remaining: 51m 23s
7000:	learn: 0.9387758	test: 0.9681742	best: 0.9681742 (7000)	total: 19m 53s	remaining: 51m 8s
7100:	learn: 0.9384334	test: 0.9681345	best: 0.9681337 (7094)	total: 20m 11s	remaining: 50m 54s
7200:	learn: 0.9380603	test: 0.9681009	best: 0.9680995 (7196)	total: 20m 29s	remaining: 50m 38s
7300:	learn: 0.9376860	test: 0.9681076	best: 0.

In [16]:
print mod.cv_scores, mod.avg_cv_score, np.std(mod.cv_scores)

[0.9611684961365539, 0.9636171979866842, 0.9527001870318789, 0.9707993591512022, 0.9673816346421713] 0.9631333749896982 0.006160540038055003


In [17]:
preds=mod.predict(test.values)

catboost_custom_estimator_regressor.py:126: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.mean(np.column_stack((est.predict(x) for est in self.fitted_models)), axis=1)


In [19]:
sub=pd.read_csv('sample_submission.csv')
res_ids=pd.read_csv('test.csv',usecols=['reservation_id']).reservation_id
sol=pd.DataFrame({'reservation_id':res_ids,'amount_spent_per_room_night_scaled':preds})
sol.to_csv('catboost_118_lr_0.02_all_cat_FE.csv',index=False)

In [20]:
res_ids=pd.read_csv('train.csv',usecols=['reservation_id']).reservation_id
oof_df=pd.DataFrame({'reservation_id':res_ids,'amount_spent_per_room_night_scaled':oof})
oof_df.to_csv('OOF_catboost_118_lr_0.02_all_cat_FE.csv',index=False)

In [16]:
cat=CatBoostRegressor(eval_metric='RMSE',n_estimators=25000,od_type='Iter',od_wait=400,colsample_bylevel=0.4,max_depth=6,
                   learning_rate=0.02)
mod=Estimator(model=cat,early_stopping_rounds=200,random_state=200,n_jobs=-1,shuffle=True,
              validation_scheme='KFold',
          n_splits=5,categorical_features_indices=cat_inds)

In [23]:
%%time
oof=mod.fit_transform(train.values,target.values)

0:	learn: 7.6430038	test: 7.6509132	best: 7.6509132 (0)	total: 154ms	remaining: 1h 4m 14s
100:	learn: 1.4444266	test: 1.4475999	best: 1.4475999 (100)	total: 12.3s	remaining: 50m 29s
200:	learn: 0.9978787	test: 0.9974773	best: 0.9974773 (200)	total: 29s	remaining: 59m 32s
900:	learn: 0.9655597	test: 0.9680263	best: 0.9680263 (900)	total: 2m 27s	remaining: 1h 5m 33s
1000:	learn: 0.9646831	test: 0.9676581	best: 0.9676581 (1000)	total: 2m 43s	remaining: 1h 5m 16s
1100:	learn: 0.9639210	test: 0.9673383	best: 0.9673383 (1100)	total: 3m	remaining: 1h 5m 11s
1200:	learn: 0.9631940	test: 0.9670583	best: 0.9670583 (1200)	total: 3m 16s	remaining: 1h 4m 55s
1300:	learn: 0.9625674	test: 0.9668468	best: 0.9668468 (1300)	total: 3m 33s	remaining: 1h 4m 56s
1400:	learn: 0.9619477	test: 0.9666340	best: 0.9666340 (1399)	total: 3m 50s	remaining: 1h 4m 48s
1500:	learn: 0.9613775	test: 0.9664497	best: 0.9664497 (1500)	total: 4m 8s	remaining: 1h 4m 44s
1600:	learn: 0.9607589	test: 0.9662706	best: 0.9662706 (

9200:	learn: 0.9306306	test: 0.9629635	best: 0.9629599 (9195)	total: 26m 53s	remaining: 46m 10s
9300:	learn: 0.9302823	test: 0.9629375	best: 0.9629375 (9300)	total: 27m 11s	remaining: 45m 54s
9400:	learn: 0.9299525	test: 0.9629379	best: 0.9629360 (9394)	total: 27m 29s	remaining: 45m 37s
9500:	learn: 0.9296052	test: 0.9629093	best: 0.9629093 (9496)	total: 27m 47s	remaining: 45m 19s
9600:	learn: 0.9292518	test: 0.9629024	best: 0.9628988 (9568)	total: 28m 4s	remaining: 45m 1s
9700:	learn: 0.9289079	test: 0.9628970	best: 0.9628894 (9653)	total: 28m 21s	remaining: 44m 44s
9800:	learn: 0.9285740	test: 0.9628785	best: 0.9628751 (9785)	total: 28m 39s	remaining: 44m 26s
9900:	learn: 0.9282393	test: 0.9628799	best: 0.9628751 (9785)	total: 28m 56s	remaining: 44m 8s
10000:	learn: 0.9279335	test: 0.9628626	best: 0.9628610 (9979)	total: 29m 15s	remaining: 43m 52s
10100:	learn: 0.9275896	test: 0.9628686	best: 0.9628596 (10076)	total: 29m 32s	remaining: 43m 34s
10200:	learn: 0.9272631	test: 0.9628639	

5000:	learn: 0.9476827	test: 0.9615503	best: 0.9615502 (4997)	total: 14m 11s	remaining: 56m 46s
5100:	learn: 0.9473018	test: 0.9615280	best: 0.9615278 (5097)	total: 14m 30s	remaining: 56m 34s
5200:	learn: 0.9469532	test: 0.9614843	best: 0.9614820 (5197)	total: 14m 49s	remaining: 56m 24s
5300:	learn: 0.9465926	test: 0.9614293	best: 0.9614290 (5299)	total: 15m 8s	remaining: 56m 17s
5400:	learn: 0.9462350	test: 0.9613825	best: 0.9613825 (5400)	total: 15m 26s	remaining: 56m 2s
5500:	learn: 0.9459076	test: 0.9613486	best: 0.9613481 (5498)	total: 15m 44s	remaining: 55m 49s
5600:	learn: 0.9455240	test: 0.9613138	best: 0.9613106 (5599)	total: 16m 3s	remaining: 55m 36s
5700:	learn: 0.9451366	test: 0.9612554	best: 0.9612542 (5697)	total: 16m 21s	remaining: 55m 23s
5800:	learn: 0.9447760	test: 0.9612127	best: 0.9612126 (5798)	total: 16m 40s	remaining: 55m 10s
5900:	learn: 0.9443890	test: 0.9611519	best: 0.9611518 (5886)	total: 16m 58s	remaining: 54m 56s
6000:	learn: 0.9440487	test: 0.9611059	best

13500:	learn: 0.9185478	test: 0.9594539	best: 0.9594465 (13487)	total: 40m 1s	remaining: 34m 5s
13600:	learn: 0.9182507	test: 0.9594696	best: 0.9594465 (13487)	total: 40m 20s	remaining: 33m 48s
13700:	learn: 0.9179521	test: 0.9594675	best: 0.9594465 (13487)	total: 40m 38s	remaining: 33m 30s
13800:	learn: 0.9176131	test: 0.9594512	best: 0.9594381 (13791)	total: 40m 57s	remaining: 33m 14s
13900:	learn: 0.9173129	test: 0.9594580	best: 0.9594381 (13791)	total: 41m 16s	remaining: 32m 57s
14000:	learn: 0.9169896	test: 0.9594482	best: 0.9594381 (13791)	total: 41m 34s	remaining: 32m 39s
14100:	learn: 0.9166538	test: 0.9594452	best: 0.9594381 (13791)	total: 41m 53s	remaining: 32m 22s
14200:	learn: 0.9163446	test: 0.9594350	best: 0.9594338 (14199)	total: 42m 11s	remaining: 32m 4s
14300:	learn: 0.9160234	test: 0.9594343	best: 0.9594299 (14223)	total: 42m 30s	remaining: 31m 47s
14400:	learn: 0.9156852	test: 0.9594397	best: 0.9594299 (14223)	total: 42m 48s	remaining: 31m 30s
14500:	learn: 0.9153781

5800:	learn: 0.9439466	test: 0.9643631	best: 0.9643594 (5781)	total: 16m 12s	remaining: 53m 38s
5900:	learn: 0.9436017	test: 0.9643321	best: 0.9643305 (5870)	total: 16m 29s	remaining: 53m 22s
6000:	learn: 0.9432446	test: 0.9642833	best: 0.9642833 (6000)	total: 16m 46s	remaining: 53m 7s
6100:	learn: 0.9428858	test: 0.9642388	best: 0.9642388 (6100)	total: 17m 3s	remaining: 52m 49s
6200:	learn: 0.9425059	test: 0.9641912	best: 0.9641882 (6185)	total: 17m 20s	remaining: 52m 33s
6300:	learn: 0.9421398	test: 0.9641697	best: 0.9641684 (6296)	total: 17m 36s	remaining: 52m 16s
6400:	learn: 0.9417420	test: 0.9641383	best: 0.9641383 (6400)	total: 17m 53s	remaining: 51m 59s
6500:	learn: 0.9413866	test: 0.9641195	best: 0.9641178 (6471)	total: 18m 10s	remaining: 51m 42s
6600:	learn: 0.9410212	test: 0.9640735	best: 0.9640735 (6600)	total: 18m 27s	remaining: 51m 27s
6700:	learn: 0.9406358	test: 0.9640414	best: 0.9640385 (6681)	total: 18m 44s	remaining: 51m 9s
6800:	learn: 0.9402782	test: 0.9639923	best

3200:	learn: 0.9532582	test: 0.9656811	best: 0.9656811 (3200)	total: 8m 15s	remaining: 56m 11s
3300:	learn: 0.9528363	test: 0.9656053	best: 0.9656045 (3299)	total: 8m 30s	remaining: 55m 58s
3400:	learn: 0.9523935	test: 0.9655355	best: 0.9655355 (3400)	total: 8m 47s	remaining: 55m 48s
3500:	learn: 0.9519994	test: 0.9654983	best: 0.9654972 (3496)	total: 9m 2s	remaining: 55m 33s
3600:	learn: 0.9516052	test: 0.9654448	best: 0.9654436 (3596)	total: 9m 19s	remaining: 55m 23s
3700:	learn: 0.9511965	test: 0.9653840	best: 0.9653835 (3695)	total: 9m 35s	remaining: 55m 12s
3800:	learn: 0.9508060	test: 0.9653394	best: 0.9653394 (3800)	total: 9m 51s	remaining: 55m
3900:	learn: 0.9504543	test: 0.9652946	best: 0.9652946 (3900)	total: 10m 7s	remaining: 54m 47s
4000:	learn: 0.9500670	test: 0.9652496	best: 0.9652477 (3996)	total: 10m 23s	remaining: 54m 33s
4100:	learn: 0.9496629	test: 0.9652034	best: 0.9652014 (4090)	total: 10m 40s	remaining: 54m 22s
4200:	learn: 0.9492479	test: 0.9651678	best: 0.965160

1500:	learn: 0.9601235	test: 0.9721802	best: 0.9721802 (1500)	total: 3m 46s	remaining: 59m 2s
1600:	learn: 0.9595864	test: 0.9719954	best: 0.9719954 (1600)	total: 4m 2s	remaining: 59m 2s
1700:	learn: 0.9590469	test: 0.9718202	best: 0.9718198 (1693)	total: 4m 17s	remaining: 58m 51s
1800:	learn: 0.9585505	test: 0.9716823	best: 0.9716823 (1800)	total: 4m 33s	remaining: 58m 46s
1900:	learn: 0.9579948	test: 0.9715026	best: 0.9715026 (1900)	total: 4m 49s	remaining: 58m 37s
2000:	learn: 0.9575222	test: 0.9713922	best: 0.9713922 (2000)	total: 5m 5s	remaining: 58m 31s
2100:	learn: 0.9571122	test: 0.9713109	best: 0.9713109 (2100)	total: 5m 21s	remaining: 58m 26s
2200:	learn: 0.9566716	test: 0.9712270	best: 0.9712253 (2199)	total: 5m 37s	remaining: 58m 16s
2300:	learn: 0.9562279	test: 0.9711209	best: 0.9711195 (2296)	total: 5m 53s	remaining: 58m 8s
2400:	learn: 0.9557818	test: 0.9709998	best: 0.9709998 (2400)	total: 6m 9s	remaining: 57m 56s
2500:	learn: 0.9553457	test: 0.9709236	best: 0.9709236 (

10200:	learn: 0.9275185	test: 0.9678291	best: 0.9678291 (10200)	total: 27m 19s	remaining: 39m 39s
10300:	learn: 0.9271683	test: 0.9678001	best: 0.9677958 (10271)	total: 27m 36s	remaining: 39m 23s
10400:	learn: 0.9268386	test: 0.9677821	best: 0.9677821 (10400)	total: 27m 52s	remaining: 39m 7s
10500:	learn: 0.9264898	test: 0.9677616	best: 0.9677581 (10486)	total: 28m 10s	remaining: 38m 53s
10600:	learn: 0.9261469	test: 0.9677389	best: 0.9677389 (10600)	total: 28m 26s	remaining: 38m 37s
10700:	learn: 0.9258427	test: 0.9677221	best: 0.9677207 (10689)	total: 28m 43s	remaining: 38m 22s
10800:	learn: 0.9255131	test: 0.9676865	best: 0.9676865 (10800)	total: 28m 59s	remaining: 38m 6s
10900:	learn: 0.9251901	test: 0.9676691	best: 0.9676661 (10877)	total: 29m 15s	remaining: 37m 50s
11000:	learn: 0.9248537	test: 0.9676582	best: 0.9676551 (10997)	total: 29m 31s	remaining: 37m 34s
11100:	learn: 0.9245143	test: 0.9676328	best: 0.9676275 (11060)	total: 29m 48s	remaining: 37m 18s
11200:	learn: 0.924211

In [24]:
print mod.cv_scores, mod.avg_cv_score, np.std(mod.cv_scores)

[0.9626907484956969, 0.9593112818965159, 0.9631693435611074, 0.9639569543100172, 0.9672556092077637] 0.9632767874942202 0.0025445577092081464


In [25]:
preds=mod.predict(test.values)

In [27]:
res_ids=pd.read_csv('train.csv',usecols=['reservation_id']).reservation_id
oof_df=pd.DataFrame({'reservation_id':res_ids,'amount_spent_per_room_night_scaled':oof})
oof_df.to_csv('OOF_catboost_118_lr_0.02_all_cat_FE_seed_200.csv',index=False)

In [28]:
sub=pd.read_csv('sample_submission.csv')
res_ids=pd.read_csv('test.csv',usecols=['reservation_id']).reservation_id
sol=pd.DataFrame({'reservation_id':res_ids,'amount_spent_per_room_night_scaled':preds})
sol.to_csv('catboost_118_lr_0.02_all_cat_FE_seed_200.csv',index=False)